## SET UP & DATA WRANGLING

In [1]:
import pandas as pd
import numpy as np 

## EDA Libraries 
import matplotlib.pyplot as plt
import seaborn as sns

## Settings
pd.set_option('display.max_columns', None) # display all columns
import warnings
warnings.filterwarnings('ignore') # ignore warnings

import hashlib # to encode personal details
import re

In [2]:
from cleaning_functions import *

In [3]:
data = pd.read_excel("typeform 15-09-2024.xlsx")

In [4]:
df = data.copy()

In [5]:
df.columns = df.columns.str.lower().str.replace(" ","_").str.replace(".","_").str.replace(":","")

In [6]:
df.shape

(1239, 24)

In [7]:
df.columns

Index(['#', '¿cómo_te_gusta_que_te_llamemos?',
       'para_empezar,_aceptas_la_política_de_privacidad_y_que_te_mandemos_chicha_de_la_buena?',
       '{{field01g9cmc9sx965qwjp9reb0q2bj}},_escribe_tu_mejor_correo',
       'escribe_tu_teléfono_(*con_prefijo,_si_no_no_te_podremos_contactar*)_para_poder_pasarte_la_información_de_nuestra_sesión_juntas❤️',
       '¿en_qué_franja_horaria_vives?',
       '-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-los_horarios_son_de_*1200_-_1400_pm_o_de_1900_a_2100_pm_*(franja_horaria_españa)\n\n-tiene_un_precio_desde*_64€_por_sesiones_de_2_horas__*pero_no_se_paga_por_sesión,_sino_que_te_comprometes_al_proceso_entero_\n\n_*esto_implica_tanto_una_inversión_económica,_como_de_tiempo_*__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__',
       'cuéntame,_¿qué_no_está_funcionando_actualmente_en_tu_vida?',
       '¿_qué_cambios_te_gustaría_ver_después_de_pasar_por_nuestra_terapia?',
       '{{field01g9cmc9sx965qwjp9re

In [8]:
df.drop(columns=["#","sí!","no,_solo_por_la_tarde!","por_las_tardes_a_partir_de_las_3_00pm","por_las_tardes_a_partir_de_las_5_00pm","por_las_tardes_a_partir_de_las_6_00pm","por_las_tardes_a_partir_de_las_7_00pm", "stage_date_(utc)", "tags"], inplace=True)

In [9]:
df = df.rename(columns={"¿cómo_te_gusta_que_te_llamemos?":"nombre",
                   "para_empezar,_aceptas_la_política_de_privacidad_y_que_te_mandemos_chicha_de_la_buena?":"politica_privacidad",
                   "{{field01g9cmc9sx965qwjp9reb0q2bj}},_escribe_tu_mejor_correo":"email",
                   "escribe_tu_teléfono_(*con_prefijo,_si_no_no_te_podremos_contactar*)_para_poder_pasarte_la_información_de_nuestra_sesión_juntas❤️":"telefono",
                   "¿en_qué_franja_horaria_vives?":"zona_horaria",
                   "-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-los_horarios_son_de_*1200_-_1400_pm_o_de_1900_a_2100_pm_*(franja_horaria_españa)\n\n-tiene_un_precio_desde*_64€_por_sesiones_de_2_horas__*pero_no_se_paga_por_sesión,_sino_que_te_comprometes_al_proceso_entero_\n\n_*esto_implica_tanto_una_inversión_económica,_como_de_tiempo_*__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__":"condiciones_terapia",
                   "cuéntame,_¿qué_no_está_funcionando_actualmente_en_tu_vida?":"problema_principal",
                   "¿_qué_cambios_te_gustaría_ver_después_de_pasar_por_nuestra_terapia?":"cambios_deseados",
                   "{{field01g9cmc9sx965qwjp9reb0q2bj}},_¿por_qué_crees_que_conecta_contigo,_nuestra_terapia_grupal,_puede_ayudarte?_":"busqueda_en_terapia",
                   "-nuestra_terapia_grupal_dura_*16_semanas_*\n\n-cada_semana_de_*1900-2100_pm_o_cada_semana_de_1200-1400_pm_*\n-se_paga_por_*todo_el_proceso*_en_cuotas_de_1/3/6_pagos_mensuales_\n\nesto_implica_una_inversión_económica_y_de_tiempo__si_esto_no_encaja_en_tus_posibilidades,_te_pedimos,_con_amor,_que_reconsideres_la_reserva__\n\n¿estás_interesada_en_continuar?":"interes_reserva",
                   "tu_tiempo_es_oro_y_el_nuestro_también,_reserva_esta_sesión_solo_*si_estás_comprometida_a_asistir__*\n\n*-esta_sesión_es_una_sesión_enfocada_a_ver_tu_caso_para_entrar_a_la_terapia_grupal_*_\n\nsin_tu_compromiso,_no_puede_haber_avance__¿trato_hecho?":"interes_sesion_personal",
                   "start_date_(utc)":"start_date",
                   "submit_date_(utc)":"submit_date"
                  })

In [10]:
df.dtypes

nombre                             object
politica_privacidad                object
email                              object
telefono                          float64
zona_horaria                       object
condiciones_terapia                object
problema_principal                 object
cambios_deseados                   object
busqueda_en_terapia                object
interes_reserva                    object
interes_sesion_personal            object
response_type                      object
start_date                 datetime64[ns]
submit_date                datetime64[ns]
network_id                         object
dtype: object

### Leyenda de columnas:

**nombre**: nombre de pila

**politica_privacidad**: acepta políticas de privacidad y consentimiento publicitario

**email**: correo electrónico

**telefono**: telefono con extensión

**zona_horaria**: 3 zonas contempladas (Europa, Latam/USA, Asia/Australia)

**condiciones_terapia**: la usuaria puede asumir condiciones y precio de terapia grupal

**problema_principal**: qué no funciona en sus vidas para buscar terapia grupal

**cambios_deseados**: qué buscan cambiar gracias a la terapia grupal

**busqueda_en_terapia**: cómo creen que la terapia grupañ les ayudará

**interes_reserva**: si están interesadas en reservar plaza

**interes_sesion_personal**: si están interesadas en realizar una sesión personalizada sobre su caso en específico de cara a entrar en la terapia grupal

**start_date**: momento en que comienzan el typeform

**submit_date**: momento en que finalizan el typeform

**network_id**: id específico de su entrada

## STANDARIZATION & NULLS

### Política de Privacidad

In [11]:
df["politica_privacidad"].value_counts()

politica_privacidad
Acepto! yes!    1069
Name: count, dtype: int64

In [12]:
df = replace_fillna(df, "politica_privacidad", replace_dict={"Acepto! yes!":"Yes"}, fill_value="No")

In [13]:
df["politica_privacidad"].value_counts()

politica_privacidad
Yes    1069
No      170
Name: count, dtype: int64

#### Teléfono

In [14]:
df["telefono"]

0       5.233226e+11
1       5.198916e+10
2       3.934979e+11
3       5.492604e+12
4       5.699234e+10
            ...     
1234             NaN
1235             NaN
1236             NaN
1237             NaN
1238             NaN
Name: telefono, Length: 1239, dtype: float64

In [15]:
df["telefono"] = df["telefono"].apply(format_phone)

In [16]:
df["telefono"].isna().sum()

372

In [17]:
df["telefono"] = df["telefono"].fillna("Sin completar")

In [18]:
df["telefono"].sample(10)

77       +34 603 40 86 99
751      +61 457 51 53 58
698      +34 671 73 60 62
19       +34 662 28 08 70
1180        Sin completar
194     +58 412 10 09 215
325     +52 556 44 66 641
513      +34 630 77 25 88
258     +39 392 66 05 478
213      +34 608 34 79 15
Name: telefono, dtype: object

### Zona horaria

In [19]:
df["zona_horaria"].value_counts()

zona_horaria
Europa                231
Latino America/USA    124
Asia/Australia          6
Name: count, dtype: int64

In [20]:
df = replace_fillna(df, "zona_horaria", replace_dict={"Europa":"Europa", "Latino America/USA":"Latam/USA", "Asia/Australia":"Asia/Australia"}, fill_value="Sin completar")

In [21]:
df["zona_horaria"].value_counts()

zona_horaria
Sin completar     878
Europa            231
Latam/USA         124
Asia/Australia      6
Name: count, dtype: int64

### Condiciones terapia

In [22]:
df["condiciones_terapia"].value_counts(dropna=False)

condiciones_terapia
NaN                                    1115
No se ajusta a mis posibilidades 🙏🏽     106
Sí se ajusta a mis posibilidades ❤️      18
Name: count, dtype: int64

In [23]:
df = replace_fillna(df, "condiciones_terapia", replace_dict={"No se ajusta a mis posibilidades 🙏🏽":"No (Latam)", "Sí se ajusta a mis posibilidades ❤️":"Yes (Latam)"}, fill_value="Yes")

In [24]:
df["condiciones_terapia"].value_counts(dropna=False)

condiciones_terapia
Yes            1115
No (Latam)      106
Yes (Latam)      18
Name: count, dtype: int64

#### Problema principal - Cambios deseados - Búsqueda en terapia

In [25]:
columnas_nan = ["problema_principal", "cambios_deseados", "busqueda_en_terapia"]

In [26]:
df[columnas_nan] = df[columnas_nan].fillna("Sin completar")

### Interés reserva

In [27]:
df["interes_reserva"].value_counts(dropna=False)

interes_reserva
Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal.                                        439
Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal, aunque me cuadraría pagar a plazos.    331
No, ahora mismo no estoy dispuesta a invertir en terapia.                                                   258
NaN                                                                                                         211
Name: count, dtype: int64

In [28]:
df = replace_fillna(df, "interes_reserva", replace_dict={"Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal.":"Yes", 
                                                         "Sí, estoy dispuesta a invertir en mí y tengo disponibilidad semanal, aunque me cuadraría pagar a plazos.":"Yes (Pago plazos)",
                                                         "No, ahora mismo no estoy dispuesta a invertir en terapia.":"No"
                                                        }, 
                                                         fill_value="Sin completar")

In [29]:
df["interes_reserva"].value_counts(dropna=False)

interes_reserva
Yes                  439
Yes (Pago plazos)    331
No                   258
Sin completar        211
Name: count, dtype: int64

#### Interés Sesión personalizada

In [30]:
df["interes_sesion_personal"].value_counts(dropna=False)

interes_sesion_personal
Sí!! Estoy dispuesta a invertirme en mí y comprometida a asistir.    820
NaN                                                                  265
No, estoy comprometida ni puedo invertir prefiero no seguir          154
Name: count, dtype: int64

In [31]:
df = replace_fillna(df, "interes_sesion_personal", replace_dict={"Sí!! Estoy dispuesta a invertirme en mí y comprometida a asistir.":"Yes", 
                                                         "No, estoy comprometida ni puedo invertir prefiero no seguir":"No"
                                                        }, 
                                                         fill_value="Sin completar")

In [32]:
df["interes_sesion_personal"].value_counts(dropna=False)

interes_sesion_personal
Yes              820
Sin completar    265
No               154
Name: count, dtype: int64

### *New column*: país

In [33]:
df_code = apply_country_code(df, "telefono", "codigo_tlf")

In [34]:
df_code = df_code.reset_index(drop=True)

In [35]:
df_code = pd.DataFrame(df_code)

In [36]:
df_code.sample(5)

,codigo_tlf
56,+34
687,+34
836,+41
511,+34
1134,Sin completar


In [37]:
df_code["pais"] = df_code["codigo_tlf"].apply(country)

In [38]:
df_code.head(5)

,codigo_tlf,pais
0,+52,México
1,+51,Perú
2,+39,Italia
3,+54,Argentina
4,+56,Chile


In [39]:
df_columns = df.columns.to_list()

In [40]:
df_columns.index("zona_horaria")

4

In [41]:
df.insert(5, "pais", df_code["pais"])

In [42]:
df.drop(columns=["codigo_tlf"], inplace=True)

## CALENDLY MERGE: SESSIONS DONE

### Set up

In [43]:
data2 = pd.read_excel("Calendly 15 sp 2024.xlsx")

In [44]:
df2 = data2.copy()

In [45]:
df2.columns = df2.columns.str.lower().str.replace(" ","_").str.replace(".","_").str.replace(":","")

In [46]:
df2.shape

(629, 51)

In [47]:
calendly = df2[["invitee_email", "start_date_&_time", "end_date_&_time"]]

In [48]:
df["sesion_completada"] = df["email"].isin(calendly["invitee_email"]).map({True: "Yes", False: "No"})

In [49]:
df["sesion_completada"].value_counts()

sesion_completada
No     700
Yes    539
Name: count, dtype: int64

### Anonymize variables 

In [50]:
df['nombre'] = df['nombre'].apply(anonymize_name)
df['email'] = df['email'].apply(anonymize_email)
df['telefono'] = df['telefono'].apply(anonymize_phone)

In [51]:
df.head(5)

,nombre,politica_privacidad,email,telefono,zona_horaria,pais,condiciones_terapia,problema_principal,cambios_deseados,busqueda_en_terapia,interes_reserva,interes_sesion_personal,response_type,start_date,submit_date,network_id,sesion_completada
0,51c75f94,Yes,d2db4125@gmail.com,+52 4eb88f07,Latam/USA,México,No (Latam),Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2024-09-15 02:31:31,2024-09-15 02:33:19,301ade9f8b,No
1,8c3a0eb7,Yes,b277736b@gmail.com,+51 40bef173,Latam/USA,Perú,No (Latam),Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2024-09-14 04:30:39,2024-09-14 04:32:26,a3dc8e3bc7,No
2,a5c59c03,Yes,fc17a6d2@gmail.com,+39 7b3288f3,Europa,Italia,Yes,A pesar del hecho que soy consciente de dónde ...,Me gustaría lograr expresarme con más segurida...,Porque ya hice terapia individual (y lo agrade...,Yes (Pago plazos),Yes,completed,2024-09-13 18:15:53,2024-09-13 18:25:03,1eab984797,No
3,6f7e8c07,Yes,e7d63ca0@gmail.com,+54 4bbe53b8,Latam/USA,Argentina,No (Latam),Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2024-09-13 05:08:20,2024-09-13 05:09:54,b46f9fdeda,No
4,707ac562,Yes,55720728@hotmail.com,+56 3a0b3c9e,Latam/USA,Chile,No (Latam),Sin completar,Sin completar,Sin completar,Sin completar,Sin completar,completed,2024-09-12 21:58:57,2024-09-12 22:00:15,ab982c39ca,No


### Sessions done

In [52]:
sesiones_si = df[df["sesion_completada"] == "Yes"]

In [53]:
sesiones_si["pais"].value_counts().head(10)

pais
España            281
Desconocido       149
México             16
Argentina          14
Colombia           11
Chile              10
Bolivia             8
Perú                6
Guatemala           5
Estados Unidos      5
Name: count, dtype: int64

In [54]:
sesiones_no = df[df["sesion_completada"] == "No"]

In [55]:
sesiones_no["pais"].value_counts().head(10)

pais
España            248
Desconocido       223
Chile              44
México             39
Argentina          35
Estados Unidos     23
Colombia           20
Guatemala          12
Italia              9
Bolivia             9
Name: count, dtype: int64

## EXPORT CLEAN CSV TO EDA

In [56]:
df.to_csv("clean_typeform.csv", index=False)